# Core Imports

In [2]:
# Custom Imports
from polysaccharide import general
from polysaccharide.general import optional_in_place
from polysaccharide.extratypes import ResidueSmarts

from polysaccharide.molutils import reactions
from polysaccharide.molutils.rdmol.rdtypes import *
from polysaccharide.molutils.rdmol import rdcompare, rdconvert, rdkdraw, rdcompare, rdprops, rdbond, rdlabels

from polysaccharide.polymer import monomer as monoutils
from polysaccharide.polymer.monomer import MonomerInfo
from polysaccharide.polymer.management import PolymerManager

from polysaccharide.polymer import building
import mbuild as mb

# Generic Imports
import re
from functools import partial, cached_property
from collections import defaultdict
from itertools import combinations, chain
from ast import literal_eval

# Numeric imports
import pandas as pd
import numpy as np

# File I/O
from pathlib import Path
import csv, json, openpyxl

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Generator, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty
from openmm.unit import Unit, Quantity

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdChemReactions

from openff.toolkit import ForceField
from openff.toolkit.topology import Topology, Molecule

# Static Paths
RAW_DATA_PATH  = Path('raw_monomer_data')
PROC_DATA_PATH = Path('processed_monomer_data')
RXN_FILES_PATH = Path('rxn_smarts')
MONO_INFO_DIR  = Path('monomer_files')

/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


# File and chemistry type definitions

In [3]:
pdb_path = Path('pdb_files')
pdb_path.mkdir(exist_ok=True)

coll_path = Path('Collections')
coll_path.mkdir(exist_ok=True)

lammps_path = Path('LAMMPS')
lammps_path.mkdir(exist_ok=True)

omm_path = Path('OpenMM')
omm_path.mkdir(exist_ok=True)

In [4]:
# defining reacting functional groups
reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}
# chemistries = ('urethane', 'NIPU')
chemistries = [i for i in reaction_pairs.keys()]

# Collating urethanes into collections and generating Interchange files

## Saving topologies to single location

In [ ]:
from tqdm.notebook import tqdm
from polysaccharide.charging.application import MolCharger


chg_method = 'Espaloma_AM1BCC'

topo_dir = Path('Topologies')
topo_dir.mkdir(exist_ok=True)

failed_topo  = defaultdict(lambda : defaultdict(list))
for chemistry in chemistries:
    chem_path = coll_path / chemistry
    chem_dir = topo_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    mgr = PolymerManager(chem_path)
    for mol_name, polymer in ( progress := tqdm(mgr.polymers.items()) ):
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        try:
            chgr = MolCharger.subclass_registry[chg_method]()
            polymer.assert_charges_for(chgr, strict=True, return_cmol=False)

            sdf_path = polymer.structure_files_chgd[chg_method]
            cmol = polymer.charged_offmol_from_sdf(chg_method)
            cmol.to_file(str(chem_dir / f'{mol_name}.sdf'), file_format='SDF')

        except AttributeError as a:
            print(a)

        except Exception as e:
            print(e)
            failed_topo[chemistry][e.__class__.__name__].append(mol_name)

## Generating Interchange dict

In [ ]:
from openforcefields.openforcefields import get_forcefield_dirs_paths

# specify forcefield
OPENFF_DIR = Path(get_forcefield_dirs_paths()[0])

# ff_name = 'openff-2.0.0.offxml'
ff_name = 'openff_unconstrained-2.0.0.offxml'
ff_path = OPENFF_DIR / ff_name
forcefield = ForceField(ff_path)

# Interchange generation
success_ics = defaultdict(defaultdict)
failed_ics  = defaultdict(list)

for chemistry in chemistries:
    chem_dir = topo_dir / chemistry
    progress = tqdm([path for path in chem_dir.iterdir()]) # unpack into list for progress bar
    for sdf_path in progress:
        mol_name = sdf_path.stem
        progress.set_postfix_str(f'{chemistry} : {mol_name}')

        offmol = Molecule.from_file(sdf_path)
        offtop = Topology.from_molecules(offmol) 

        try:
            ic = forcefield.create_interchange(offtop, charge_from_molecules=[offmol])
            success_ics[chemistry][mol_name] = ic
        except Exception as e:
            print(e)
            failed_ics[e.__class__.__name__].append(sdf_path)

for err_name, err_list in failed_ics.items():
    for sdf_path in err_list:
        sdf_path.unlink() # delete dud files

# Running OpenMM simulations

## Defining utility functions

In [ ]:
from openmm import XmlSerializer
from openmm import System, Context, State
from openmm import Integrator, Force
from openmm.app import Simulation
from openmm.unit import nanometer

from openff.interchange import Interchange
from openff.units import unit as offunit

from polysaccharide.simulation.ensemble import EnsembleSimulationFactory
from polysaccharide.simulation.records import SimulationParameters

DEFAULT_STATE_PARAMS : dict[str, bool] = {
    'getPositions'  : True,
    'getVelocities' : True,
    'getForces'     : True,
    'getEnergy'     : True,
    'getParameters' : True,
    'getParameterDerivatives' : False,
    'getIntegratorParameters' : False
}


def serialize_state_and_sys(sim : Simulation, out_dir : Path, out_name : str, state_params : dict[str, bool]=DEFAULT_STATE_PARAMS) -> None:
    '''For saving State and System info of a Simulation to disc'''
    sim_dict = {
        'system' : sim.system,
        'state' : sim.context.getState(**state_params)
    }
    
    for affix, save_data in sim_dict.items():
        save_path = out_dir / f'{out_name}_{affix}.xml'
        save_path.touch()

        with save_path.open('w') as file:
            file.write( XmlSerializer.serialize(save_data) )

def apply_state_to_context(state : State, context : Context) -> None:
    '''For applying saved State data to an existing OpenMM Simulation'''
    context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
    context.setPositions(state.getPositions())
    context.setVelocities(state.getVelocities())
    context.setTime(state.getTime())

    context.reinitialize(preserveState=True)    

def load_openmm_system(sys_path : Path, extra_forces : Optional[Union[Force, Iterable[Force]]]=None, sep_force_grps : bool=True, remove_constrs : bool=False) -> System:
    '''Load and configure a serialized OpenMM system, with optional additional parameters'''
    assert(sys_path.suffix == '.xml')
    with sys_path.open('r') as file:
        ommsys = XmlSerializer.deserialize(file.read())

    if extra_forces: # deliberately sparse to handle both Nonetype and empty list
        for force in extra_forces: 
            ommsys.addForce(force)

    if sep_force_grps:
        for i, force in enumerate(ommsys.getForces()):
            force.setForceGroup(i)

    if remove_constrs:
        for i in range(ommsys.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
            ommsys.removeConstraint(i)

    return ommsys

def create_simulation2(interchange : Interchange, integrator : Integrator, forces : Optional[Iterable[Force]]=None,
                        sep_force_grps : bool=True, remove_constrs : bool=True, combine_nonbonded_forces : bool=True) -> Simulation:
    '''Specifies configuration for an OpenMM Simulation - Interchange load alows many routes for creation'''
    openmm_sys = interchange.to_openmm(combine_nonbonded_forces=combine_nonbonded_forces) 
    openmm_top = interchange.topology.to_openmm()
    openmm_pos = interchange.positions.m_as(offunit.nanometer) * nanometer

    if forces: # deliberately sparse to handle both Nonetype and empty list
        for force in forces: 
            openmm_sys.addForce(force)

    if sep_force_grps:
        for i, force in enumerate(openmm_sys.getForces()):
            force.setForceGroup(i)

    if remove_constrs:
        for i in range(openmm_sys.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
            openmm_sys.removeConstraint(i)

    simulation = Simulation(openmm_top, openmm_sys, integrator)
    simulation.context.setPositions(openmm_pos)

    return simulation

def openmm_sim_from_files(top_file : Path, sys_file : Path, state_file : Path, sim_params : SimulationParameters, sep_force_grps : bool=True, remove_constrs : bool=False, box_vectors : Optional[np.ndarray]=None) -> Simulation:
    '''Load an OpenMM simulation object from serialized Topology, System, and State'''
    offmol = Molecule.from_file(top_file)
    offtop = Topology.from_molecules(offmol)
    if box_vectors is not None:
        offtop.box_vectors = box_vectors
    ommtop = offtop.to_openmm()


    # define ensemble-specific forces and Integrator
    ens_fac = EnsembleSimulationFactory.subclass_registry[sim_params.ensemble.upper()]()
    integrator = ens_fac.integrator(sim_params)
    forces     = ens_fac.forces(    sim_params)

    # load and configure System
    ommsys = load_openmm_system(sys_file, extra_forces=forces, sep_force_grps=sep_force_grps, remove_constrs=remove_constrs)

    # putting it all together into a Simulation
    return Simulation(
        topology=ommtop,
        system=ommsys,
        integrator=integrator,
        state=state_file
    )

# load and apply State
# with state_file.open('r') as file:
#     ommstate = XmlSerializer.deserialize(file.read())
# apply_state_to_context(ommstate, sim.context)

## Manually create OpenMM sims from Interchange

In [ ]:
# specifying simulation and ensemble parameters
from openmm.app import Simulation
from openmm import NonbondedForce, CustomNonbondedForce
from openmm import MonteCarloBarostat, LangevinMiddleIntegrator

from openmm.unit import atmosphere, kelvin
from openmm.unit import femtosecond, picosecond
from openff.units import unit as offunit

# Box sizes
BOX_VECS = np.eye(3) * 10 * nanometer

# Long-range parameters
CUTOFF = 2.0 * nanometer
# CUTOFF_METHOD = NonbondedForce.NoCutoff
# CUTOFF_METHOD = NonbondedForce.CutoffNonPeriodic
CUTOFF_METHOD = NonbondedForce.CutoffPeriodic

DISPERSION = False
SWITCHING  = False

# Thermodynamic/integrator parameters
T = 300*kelvin
P = 1*atmosphere

timestep = 2*femtosecond
friction = 1*picosecond**-1

# ======================================

force_names = (
    'vdW pairwise',
    'Electrostatic',
    '1-4 LJ',
    '1-4 Coulomb',
    'Torsion',
    'Angle',
    'Bond'
)

# looping over all urethanes
omm_sims = defaultdict(defaultdict)
for chemistry, ic_dict in success_ics.items():
    for mol_name, interchange in (progress := tqdm(ic_dict.items())):
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        
        # specifying thermo/baro to determine ensemble
        integrator = LangevinMiddleIntegrator(T, friction, timestep)
        # extra_forces = [MonteCarloBarostat(P, T, baro_freq)]
        extra_forces = None

        # loading OpenMM sim components from Interchange
        omm_top = interchange.topology.to_openmm()
        omm_sys = interchange.to_openmm(combine_nonbonded_forces=False)
        omm_pos = interchange.positions.m_as(offunit.nanometer)

        ## Setting box vectors for periodic forces
        omm_top.setPeriodicBoxVectors(BOX_VECS)
        omm_sys.setDefaultPeriodicBoxVectors(*BOX_VECS)

        # configuring bound Force objects
        if extra_forces:
            for force in extra_forces:
                omm_sys.addForce(force)

        ## number all forces into separate force groups for separability
        for i, force in enumerate(omm_sys.getForces()):
            force.setForceGroup(i)

        ## Add labels to default forces
        for force, name in zip(omm_sys.getForces(), force_names):
            force.setName(name)

        ## reconfiguring non-bonded forces
        ### Custom nonbonded
        nonbond_custom = omm_sys.getForce(0)
        assert(isinstance(nonbond_custom, CustomNonbondedForce))

        nonbond_custom.setCutoffDistance(CUTOFF)
        nonbond_custom.setUseSwitchingFunction(SWITCHING)
        nonbond_custom.setNonbondedMethod(CUTOFF_METHOD)
        nonbond_custom.setUseLongRangeCorrection(DISPERSION)
 
        ### Default nonbonded
        nonbond = omm_sys.getForce(1)
        assert(isinstance(nonbond, NonbondedForce))

        nonbond.setCutoffDistance(CUTOFF)
        nonbond.setNonbondedMethod(CUTOFF_METHOD)
        nonbond.setUseSwitchingFunction(SWITCHING)
        nonbond.setUseDispersionCorrection(DISPERSION)

        # create and register simulation
        sim = Simulation(omm_top, omm_sys, integrator)
        sim.context.setPositions(omm_pos)
        omm_sims[chemistry][mol_name] = sim

## Serialize OpenMM simulations to file

In [ ]:
# specifying simulation and ensemble parameters
omm_dir = Path('OpenMM_newest')
omm_dir.mkdir(exist_ok=True)

# looping over all urethanes
for chemistry, sim_dict in omm_sims.items():
    chem_dir = omm_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    for mol_name, sim in (progress := tqdm(sim_dict.items())):
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        interchange = success_ics[chemistry][mol_name]

        # creating directories
        mol_dir = chem_dir / mol_name 
        mol_dir.mkdir(exist_ok=True)

        # serialize Topology, System, and State for reloading
        sdf_out_path = mol_dir / f'{mol_name}_topology.sdf'
        sdf_out_path.touch()
        for mol in interchange.topology.molecules: # use OpenFF format for saving Molecules (much more convenient to work with)
            mol.to_file(str(sdf_out_path), file_format='SDF')
        serialize_state_and_sys(sim, out_dir=mol_dir, out_name=mol_name)

## Loading simulations from file

In [ ]:
from polysaccharide.simulation.records import SimulationParameters


CUTOFF = 2.0 * nanometer
BOX_VECS = np.eye(3) * 10 * nanometer

# parameters
sep_force_grps : bool = True
remove_constrs : bool = False

# paths
# omm_dir = Path('OpenMM')
omm_dir = Path('OpenMM_unconstrained')
sp_path = Path('debug_sim_NVT.json')

# iterate over serialized directory tree and load
sim_params = SimulationParameters.from_file(sp_path)

omm_sims = defaultdict(defaultdict)
for subdir in omm_dir.iterdir():
    if subdir.is_dir():
        chemistry = subdir.name
        for mol_dir in subdir.iterdir():
            mol_name = mol_dir.name

            state_file = mol_dir / f'{mol_name}_state.xml'
            sys_file   = mol_dir / f'{mol_name}_system.xml'
            top_file   = mol_dir / f'{mol_name}_topology.sdf'

            sim = openmm_sim_from_files(
                top_file,
                sys_file,
                state_file,
                sim_params,
                sep_force_grps=sep_force_grps,
                remove_constrs=remove_constrs,
            )

            omm_sims[chemistry][mol_name] = sim

## Evaluating starting structure energies

In [ ]:
from openmm.unit import kilojoule_per_mole, kilocalorie_per_mole


NULL_ENERGY = 0.0*kilojoule_per_mole
PRECISION : int = 4

data_dicts = []
for chemistry, mol_dict in omm_sims.items():
    for mol_name, sim in (progress := tqdm(mol_dict.items())):
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        
        # extract total and component energies from OpenMM force groups
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }
        omm_energies = {}

        ## Total Potential
        overall_state = sim.context.getState(getEnergy=True) # get total potential energy
        PE = overall_state.getPotentialEnergy()
        omm_energies['Potential'] = PE

        ## Total Kinetic (to verify no integration is being done)
        KE = overall_state.getKineticEnergy()
        omm_energies['Kinetic'] = KE
        assert(KE == NULL_ENERGY)

        ## Individual force contributions
        for i, force in enumerate(sim.system.getForces()):
            state = sim.context.getState(getEnergy=True, groups={i})
            omm_energies[force.getName()] = state.getPotentialEnergy()

        # reformat to desired units and precision
        omm_energies_kcal = {}
        for contrib_name, energy_kj in omm_energies.items():
            energy_kcal = energy_kj.in_units_of(kilocalorie_per_mole)
            omm_energies_kcal[f'{contrib_name} ({energy_kcal.unit.get_symbol()})'] = round(energy_kcal._value, PRECISION)

        # compile data
        data_dict = {**data_dict, **omm_energies_kcal}
        data_dicts.append(data_dict)

omm_table = pd.DataFrame.from_records(data_dicts)
omm_table.sort_values('Molecule', inplace=True)
omm_table.set_index(['Chemistry', 'Molecule'], inplace=True)
omm_table.to_csv(omm_dir / f'{omm_dir.name}_PEs.csv')

# Comparing energies

In [5]:
pd.options.display.float_format = '{:.4f}'.format # disable scientific notation

omm_path = Path('OpenMM_newest/OpenMM_newest_PEs.csv')
omm_table = pd.read_csv(omm_path, index_col=(0,1))
omm_table = omm_table.sort_index(axis=1)

lmp_path = Path('LAMMPS/LAMMPS_PEs.csv')
lmp_table = pd.read_csv(lmp_path, index_col=(0,1))
lmp_table['Torsion (kcal/mol)'] = lmp_table['Dihedral (kcal/mol)'] + lmp_table['Impropers (kcal/mol)']

lmp_table.drop(columns=['Dihedral (kcal/mol)', 'Impropers (kcal/mol)'], inplace=True)
lmp_table.drop(columns=['Ranged k-space (kcal/mol)', 'vdW ranged (kcal/mol)', 'Total (kcal/mol)', 'Pairwise (kcal/mol)'], inplace=True)
lmp_table = lmp_table.sort_index(axis=1)

In [6]:
omm_table

1-4 Coulomb (kcal/mol)  1-4 LJ (kcal/mol)  \
Chemistry Molecule                                                 
NIPU      NIPU_11                    205.6854           447.5183   
          NIPU_2                    -715.4404           464.7511   
urethane  urethane_0               -6900.4149           621.6685   
          urethane_1               -2090.4597           623.4489   
          urethane_10              -1430.2346           835.3169   
          urethane_14               -992.3145           525.7142   
          urethane_15               -310.1130           366.5830   
          urethane_17              -7030.1549           549.6743   
          urethane_2               -1186.9846           509.9049   
          urethane_22              -7357.8457          1014.2406   
          urethane_24                -65.9386           215.5036   
          urethane_29              -2438.1617           332.5200   
          urethane_35              -1804.3918           767.2047   
          urethane_41               -954.2175           554.9805   
          urethane_42              -7307.9640           524.9001   
          urethane_44              -3265.4415           432.4624   
          urethane_45              -5835.5197           772.0958   
          urethane_47              -2809.9900           569.2876   
          urethane_48              -6408.7810           633.2529   
          urethane_51               -858.4702           533.7714   
          urethane_52               -920.8460           345.4618   
          urethane_6               -2478.5492           318.4693   
          urethane_8                -827.2775           589.3206   

                       Angle (kcal/mol)  Bond (kcal/mol)  \
Chemistry Molecule                                         
NIPU      NIPU_11              924.4440         936.1703   
          NIPU_2              1731.2338        1054.8592   
urethane  urethane_0          4417.8422         745.0805   
          urethane_1           731.2801         591.9922   
          urethane_10         1165.3718         585.7152   
          urethane_14         1333.2802         606.2830   
          urethane_15          814.8633         581.8832   
          urethane_17         1743.4136         660.9872   
          urethane_2          1287.5645         604.7534   
          urethane_22         6848.7392         713.5582   
          urethane_24          720.4697         573.5574   
          urethane_29          606.1829         574.1365   
          urethane_35          649.4389         577.7948   
          urethane_41         1797.4608         633.1721   
          urethane_42         1738.2827         668.5361   
          urethane_44         1000.4309         573.2831   
          urethane_45         4355.3222         722.5862   
          urethane_47         1178.6489         617.1913   
          urethane_48         2010.1302         688.4367   
          urethane_51          797.4400         575.3877   
          urethane_52          882.3442         580.9579   
          urethane_6          1128.0501         589.4641   
          urethane_8           710.3142         570.0546   

                       Electrostatic (kcal/mol)  Kinetic (kcal/mol)  \
Chemistry Molecule                                                    
NIPU      NIPU_11                     -701.3613              0.0000   
          NIPU_2                      -629.1520              0.0000   
urethane  urethane_0                  2031.9201              0.0000   
          urethane_1                   569.9974              0.0000   
          urethane_10                  283.0705              0.0000   
          urethane_14                  153.5438              0.0000   
          urethane_15                 -308.6600              0.0000   
          urethane_17                 2615.6603              0.0000   
          urethane_2                   400.4940              0.0000   
          urethane_22                 14

In [7]:
lmp_table

Angle (kcal/mol)  Bond (kcal/mol)  Coulomb (kcal/mol)  \
Chemistry Molecule                                                             
NIPU      NIPU_11              924.4441         936.1614           -817.7800   
          NIPU_2              1731.2339        1054.8606          -1858.4350   
urethane  urethane_0          4417.8420         745.0806          -4124.3930   
          urethane_1           731.2807         591.9917          -1311.1273   
          urethane_10         1165.3723         585.7113          -1147.8505   
          urethane_14         1333.2805         606.2828           -789.8303   
          urethane_15          814.8639         581.8827           -650.7598   
          urethane_17         1743.4138         660.9857          -3544.3699   
          urethane_2          1287.5646         604.7534           -724.7406   
          urethane_22         6848.7400         713.5584          -5443.9864   
          urethane_24          720.4702         573.5574           -494.6371   
          urethane_29          606.1828         574.1360          -1441.9564   
          urethane_35          649.4389         577.7903          -1750.4180   
          urethane_41         1797.4613         633.1740           -771.2413   
          urethane_42         1738.2825         668.5365          -3780.1674   
          urethane_44         1000.4310         573.2828          -1913.7511   
          urethane_45         4355.3228         722.5881          -3966.8073   
          urethane_47         1178.6491         617.1921          -1908.6936   
          urethane_48         2010.1293         688.4420          -3624.6179   
          urethane_51          797.4399         575.3868           -819.3621   
          urethane_52          882.3444         580.9560           -810.4222   
          urethane_6          1128.0496         589.4681          -1784.8192   
          urethane_8           710.3152         570.0580           -744.1771   

                       Kinetic (kcal/mol)  Potential (kcal/mol)  \
Chemistry Molecule                                                
NIPU      NIPU_11                  0.0000            25796.0698   
          NIPU_2                   0.0000            15502.0279   
urethane  urethane_0               0.0000             3253.1577   
          urethane_1               0.0000             2518.2398   
          urethane_10              0.0000             2611.1150   
          urethane_14              0.0000             3403.3569   
          urethane_15              0.0000            20892.7496   
          urethane_17              0.0000              531.4367   
          urethane_2               0.0000          2270864.1618   
          urethane_22              0.0000          1013610.8417   
          urethane_24              0.0000             1347.8329   
          urethane_29              0.0000              544.5518   
          urethane_35              0.0000            14092.8431   
          urethane_41              0.0000             5690.2783   
          urethane_42              0.0000             -305.8911   
          urethane_44              0.0000            14965.5484   
          urethane_45              0.0000             3666.3564   
          urethane_47              0.0000             1191.6175   
          urethane_48              0.0000     294533593910.3697   
          urethane_51              0.0000             1830.1584   
          urethane_52              0.0000             2666.0512   
          urethane_6               0.0000              670.9856   
          urethane_8               0.0000             1832.9615   

                       Torsion (kcal/mol)  vdW pairwise (kcal/mol)  
Chemistry Molecule                                                  
NIPU      NIPU_11                307.5251               24445.7192  
          NIPU_2                 403.6802               14170.6882  
urethane  urethane_0             465.7065                1748.9215  
 

In [8]:
common_cols = ['Angle (kcal/mol)', 'Bond (kcal/mol)', 'Torsion (kcal/mol)', 'Kinetic (kcal/mol)']

omm_redux = omm_table.drop(columns=common_cols)
lmp_redux = lmp_table.drop(columns=common_cols)

In [9]:
omm_table[common_cols] - lmp_table[common_cols]

Angle (kcal/mol)  Bond (kcal/mol)  Torsion (kcal/mol)  \
Chemistry Molecule                                                             
NIPU      NIPU_11               -0.0001           0.0089              0.0000   
          NIPU_2                -0.0001          -0.0014             -0.0000   
urethane  urethane_0             0.0002          -0.0001              0.0001   
          urethane_1            -0.0006           0.0005             -0.0000   
          urethane_10           -0.0005           0.0039             -0.0001   
          urethane_14           -0.0003           0.0002              0.0002   
          urethane_15           -0.0006           0.0005              0.0000   
          urethane_17           -0.0002           0.0015              0.0001   
          urethane_2            -0.0001           0.0000              0.0000   
          urethane_22           -0.0008          -0.0002              0.0000   
          urethane_24           -0.0005          -0.0000             -0.0000   
          urethane_29            0.0001           0.0005              0.0000   
          urethane_35           -0.0000           0.0045              0.0000   
          urethane_41           -0.0005          -0.0019              0.0001   
          urethane_42            0.0002          -0.0004             -0.0001   
          urethane_44           -0.0001           0.0003              0.0001   
          urethane_45           -0.0006          -0.0019              0.0000   
          urethane_47           -0.0002          -0.0008              0.0001   
          urethane_48            0.0009          -0.0053              0.0005   
          urethane_51            0.0001           0.0009              0.0000   
          urethane_52           -0.0002           0.0019             -0.0001   
          urethane_6             0.0005          -0.0040              0.0001   
          urethane_8            -0.0010          -0.0034              0.0000   

                       Kinetic (kcal/mol)  
Chemistry Molecule                         
NIPU      NIPU_11                  0.0000  
          NIPU_2                   0.0000  
urethane  urethane_0               0.0000  
          urethane_1               0.0000  
          urethane_10              0.0000  
          urethane_14              0.0000  
          urethane_15              0.0000  
          urethane_17              0.0000  
          urethane_2               0.0000  
          urethane_22              0.0000  
          urethane_24              0.0000  
          urethane_29              0.0000  
          urethane_35              0.0000  
          urethane_41              0.0000  
          urethane_42              0.0000  
          urethane_44              0.0000  
          urethane_45              0.0000  
          urethane_47              0.0000  
          urethane_48              0.0000  
          urethane_51              0.0000  
          urethane_52              0.0000  
          urethane_6               0.0000  
          urethane_8               0.0000

In [10]:
omm_redux

1-4 Coulomb (kcal/mol)  1-4 LJ (kcal/mol)  \
Chemistry Molecule                                                 
NIPU      NIPU_11                    205.6854           447.5183   
          NIPU_2                    -715.4404           464.7511   
urethane  urethane_0               -6900.4149           621.6685   
          urethane_1               -2090.4597           623.4489   
          urethane_10              -1430.2346           835.3169   
          urethane_14               -992.3145           525.7142   
          urethane_15               -310.1130           366.5830   
          urethane_17              -7030.1549           549.6743   
          urethane_2               -1186.9846           509.9049   
          urethane_22              -7357.8457          1014.2406   
          urethane_24                -65.9386           215.5036   
          urethane_29              -2438.1617           332.5200   
          urethane_35              -1804.3918           767.2047   
          urethane_41               -954.2175           554.9805   
          urethane_42              -7307.9640           524.9001   
          urethane_44              -3265.4415           432.4624   
          urethane_45              -5835.5197           772.0958   
          urethane_47              -2809.9900           569.2876   
          urethane_48              -6408.7810           633.2529   
          urethane_51               -858.4702           533.7714   
          urethane_52               -920.8460           345.4618   
          urethane_6               -2478.5492           318.4693   
          urethane_8                -827.2775           589.3206   

                       Electrostatic (kcal/mol)  Potential (kcal/mol)  \
Chemistry Molecule                                                      
NIPU      NIPU_11                     -701.3613            26118.2444   
          NIPU_2                      -629.1520            16015.8939   
urethane  urethane_0                  2031.9201             2509.0481   
          urethane_1                   569.9974             2308.9098   
          urethane_10                  283.0705             2611.8066   
          urethane_14                  153.5438             3354.4142   
          urethane_15                 -308.6600            20924.9130   
          urethane_17                 2615.6603             -338.6883   
          urethane_2                   400.4940          2270775.0956   
          urethane_22                 1490.6013          1013185.1099   
          urethane_24                 -292.1903             1484.3386   
          urethane_29                  786.2563              334.6035   
          urethane_35                  -21.9853            14016.8452   
          urethane_41                  117.0474             5624.2139   
          urethane_42                 2649.7443            -1183.9486   
          urethane_44                 1165.1918            14779.0569   
          urethane_45                 1521.2731             3318.9333   
          urethane_47                  658.7929              949.1032   
          urethane_48                 1825.7297     294535481269.5984   
          urethane_51                    6.3879             1797.4339   
          urethane_52                   71.3160             2626.9489   
          urethane_6                   687.6239              664.8717   
          urethane_8                    12.3240             1762.1819   

                       vdW pairwise (kcal/mol)  
Chemistry Molecule                              
NIPU      NIPU_11                   23998.2635  
          NIPU_2                    13705.9623  
urethane  urethane_0                 1127.2451  
          urethane_1                 1701.8747  
          urethane_10                 821.7283  
          urethane_14                1459.3529  
          urethane_15               19510.5555  
          urethane_17                 781.1094  
      

In [11]:
lmp_redux

Coulomb (kcal/mol)  Potential (kcal/mol)  \
Chemistry Molecule                                                
NIPU      NIPU_11               -817.7800            25796.0698   
          NIPU_2               -1858.4350            15502.0279   
urethane  urethane_0           -4124.3930             3253.1577   
          urethane_1           -1311.1273             2518.2398   
          urethane_10          -1147.8505             2611.1150   
          urethane_14           -789.8303             3403.3569   
          urethane_15           -650.7598            20892.7496   
          urethane_17          -3544.3699              531.4367   
          urethane_2            -724.7406          2270864.1618   
          urethane_22          -5443.9864          1013610.8417   
          urethane_24           -494.6371             1347.8329   
          urethane_29          -1441.9564              544.5518   
          urethane_35          -1750.4180            14092.8431   
          urethane_41           -771.2413             5690.2783   
          urethane_42          -3780.1674             -305.8911   
          urethane_44          -1913.7511            14965.5484   
          urethane_45          -3966.8073             3666.3564   
          urethane_47          -1908.6936             1191.6175   
          urethane_48          -3624.6179     294533593910.3697   
          urethane_51           -819.3621             1830.1584   
          urethane_52           -810.4222             2666.0512   
          urethane_6           -1784.8192              670.9856   
          urethane_8            -744.1771             1832.9615   

                       vdW pairwise (kcal/mol)  
Chemistry Molecule                              
NIPU      NIPU_11                   24445.7192  
          NIPU_2                    14170.6882  
urethane  urethane_0                 1748.9215  
          urethane_1                 2325.3186  
          urethane_10                1657.0433  
          urethane_14                1985.0695  
          urethane_15               19876.9603  
          urethane_17                1330.7853  
          urethane_2              2269441.8774  
          urethane_22             1010925.5020  
          urethane_24                 290.6818  
          urethane_29                 611.9376  
          urethane_35               14521.0710  
          urethane_41                3696.7870  
          urethane_42                 727.4642  
          urethane_44               15162.1420  
          urethane_45                1995.5494  
          urethane_47                 995.9925  
          urethane_48        294533594473.7455  
          urethane_51                1091.5963  
          urethane_52                1808.2313  
          urethane_6                  414.7321  
          urethane_8                 1139.1170

In [15]:
compare_cols = ['vdW pairwise (kcal/mol)', 'Potential (kcal/mol)']

(omm_redux[compare_cols] - lmp_redux[compare_cols]) / omm_redux[compare_cols]*100

vdW pairwise (kcal/mol)  Potential (kcal/mol)
Chemistry Molecule                                                  
NIPU      NIPU_11                      -1.8645                1.2335
          NIPU_2                       -3.3907                3.2085
urethane  urethane_0                  -55.1501              -29.6570
          urethane_1                  -36.6328               -9.0662
          urethane_10                -101.6534                0.0265
          urethane_14                 -36.0240               -1.4591
          urethane_15                  -1.8780                0.1537
          urethane_17                 -70.3712              256.9103
          urethane_2                   -0.0236               -0.0039
          urethane_22                  -0.1007               -0.0420
          urethane_24                -286.6679                9.1964
          urethane_29                -119.0046              -62.7454
          urethane_35                  -5.5784               -0.5422
          urethane_41                 -17.6694               -1.1746
          urethane_42                -259.1364               74.1635
          urethane_44                  -2.9359               -1.2619
          urethane_45                 -63.1054              -10.4679
          urethane_47                -133.4202              -25.5519
          urethane_48                   0.0006                0.0006
          urethane_51                 -95.6898               -1.8206
          urethane_52                 -23.6167               -1.4885
          urethane_6                 -330.8516               -0.9196
          urethane_8                 -107.1885               -4.0166

In [13]:
omm_redux['1-4 Coulomb (kcal/mol)'] + omm_redux['Electrostatic (kcal/mol)'] - lmp_redux['Coulomb (kcal/mol)']

Chemistry  Molecule   
NIPU       NIPU_11        322.1041
           NIPU_2         513.8426
urethane   urethane_0    -744.1018
           urethane_1    -209.3350
           urethane_10      0.6864
           urethane_14    -48.9404
           urethane_15     31.9868
           urethane_17   -870.1247
           urethane_2     -61.7500
           urethane_22   -423.2580
           urethane_24    136.5082
           urethane_29   -209.9490
           urethane_35    -75.9591
           urethane_41    -65.9288
           urethane_42   -878.0523
           urethane_44   -186.4986
           urethane_45   -347.4393
           urethane_47   -242.5035
           urethane_48   -958.4334
           urethane_51    -32.7202
           urethane_52    -39.1078
           urethane_6      -6.1061
           urethane_8     -70.7764
dtype: float64

## Comparing energies from constrained and unconstrained FFs

In [ ]:
e_tables = {
    platform : pd.read_csv(Path(platform) / f'{platform}_PEs.csv', index_col=(0, 1))
        for platform in ('LAMMPS', 'OpenMM', 'OpenMM_unconstrained')
}

In [ ]:
cols = [
    'Nonbonded (kcal/mol)',
    'Bond (kcal/mol)',
    'Angle (kcal/mol)',
    'Torsion (kcal/mol)',
    'Potential (kcal/mol)',
    'Kinetic (kcal/mol)',
]

e_tables.pop('OpenMM')
for src, tab in e_tables.items():
    print(src)
    display(tab[cols])

In [ ]:
diff = e_tables['OpenMM_unconstrained'] - e_tables['LAMMPS']
diff

In [ ]:
diff['Torsion (kcal/mol)'] - e_tables['LAMMPS']['Impropers (kcal/mol)']

In [ ]:
df_unconstr.loc[[('urethane', 'urethane_0'), ('urethane', 'urethane_1'), ('urethane', 'urethane_41')]]

In [ ]:
nonbond = sim.system.getForce(0)
nonbond.getCutoffDistance()

In [ ]:
nonbond.getName()

In [ ]:
data_dicts = []
for chemistry, mol_dict in omm_sims.items():
    print(chemistry)
    for mol_name, sim in mol_dict.items():
        print(mol_name)
        nonbond = sim.system.getForce(0)
        assert(nonbond.getName() == 'NonbondedForce')
        print(nonbond.getCutoffDistance())

In [ ]:
import matplotlib.pyplot as plt

bond_diffs = e_diff['HarmonicBond (kcal/mol)'].to_numpy()
plt.hist(bond_diffs, bins=20)
plt.xlim(bond_diffs.min(), bond_diffs.max())
plt.xlabel(f'{general.GREEK_UPPER["delta"]}E_bond const vs unconst')
plt.ylabel('Count')

## Comparing ParmEd energy decomposition to native OpenMM force-group-based decomposition

In [ ]:
import parmed
from openmm.openmm import Force

NULL_ENERGY = 0.0*kilojoule_per_mole

sim = omm_sims['urethane']['urethane_41']
# assign and initialize unique force groups for simulation
for i, force in enumerate(sim.system.getForces()):
    force.setForceGroup(i)
    # print(force.getName(), force.getForceGroup())
sim.context.reinitialize(preserveState=True) # need to reinitialize to get force labelling changes to "stick"

# energies from OpenMM force groups
print('\nOpenMM:')
print('='*30)
omm_energies = {}

## extract total energies for state
overall_state = sim.context.getState(getEnergy=True) # get total potential energy
PE = overall_state.getPotentialEnergy()
omm_energies['Total Potential Energy'] = PE

KE = overall_state.getKineticEnergy()
assert(KE == NULL_ENERGY)

for i, force in enumerate(sim.system.getForces()):
    state = sim.context.getState(getEnergy=True, groups={i})
    force_name = force.getName().removesuffix('Force')
    pe = state.getPotentialEnergy()

    omm_energies[force_name] = pe
    print(f'{force_name} : {pe}')

## converting name to match with ParmEd for comparison
namemap = {
    'Nonbonded' : 'bond',
    'PeriodicTorsion' : 'angle',
    'HarmonicAngle' : 'dihedral',
    'HarmonicBond' : 'urey_bradley',
    'Total Potential Energy' : 'total'
}
compat_omm_energies = {
    namemap[contrib] : energy
        for contrib, energy in omm_energies.items()
}

total = sum(omm_energies.values(), start=NULL_ENERGY) # need "seed" to have Quantity datatype to sum
print(f'{general.GREEK_UPPER["delta"]}E_contrib: ', PE - total)

# ParmEd energy decomposition
print('\nParmEd:')
print('='*30)
parm_energies = {}
parm_struct = parmed.openmm.load_topology(sim.topology, sim.system)
for contrib, energy_val in parmed.openmm.energy_decomposition(parm_struct, sim.context).items():
    parm_energies[contrib] = energy = energy_val*kilocalorie_per_mole # assign proper units
    print(contrib, energy.in_units_of(kilojoule_per_mole))

## Minimizing and running single integration step, then evaluating energies from reporter

In [ ]:
import re
from openmm.unit import kilojoule_per_mole, kilocalorie_per_mole

from polysaccharide.simulation import preparation
from polysaccharide.simulation.records import SimulationParameters
from polysaccharide.simulation.ensemble import EnsembleSimulationFactory

STRIP_BEFORE_PARENS = re.compile(r'(.*?)(?=\s*\(.*\))')
PRECISION = 3 # number of decimals to round reported energies to

omm_dir = Path('OpenMM_no_sim')
omm_dir.mkdir(exist_ok=True)


# selecting simulation parameters and ensemble
sp_path = Path('debug_sim_NVT.json')
sim_params = SimulationParameters.from_file(sp_path)
ens_fac = EnsembleSimulationFactory.subclass_registry[sim_params.ensemble.upper()]()

data_by_mol = []
omm_sims = defaultdict(defaultdict)
# looping over all urethanes
for chemistry, ic_dict in success_ics.items():
    chem_dir = omm_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    for mol_name, interchange in ic_dict.items():
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }

        # creating directories
        mol_dir = chem_dir / mol_name 
        mol_dir.mkdir(exist_ok=True)

        sim_file_dir = mol_dir / f'{mol_name}_sim'
        sim_file_dir.mkdir(exist_ok=True, parents=True)

        # creating simulation and associated files
        sim = ens_fac.create_simulation(interchange, sim_params)
        sim_paths = preparation.prepare_simulation_paths(output_folder=sim_file_dir, output_name=mol_name, sim_params=sim_params)
        reporters = preparation.prepare_simulation_reporters(sim_paths, sim_params)
        preparation.config_simulation(sim, reporters, checkpoint_path=sim_paths.checkpoint)

        # energy min and single-step integration
        sim.minimizeEnergy()
        sim.step(1)

        # extracting energies

        state_data = pd.read_csv(sim_paths.state_data)
        energies = {}
        for key in ('Potential Energy (kJ/mole)', 'Kinetic Energy (kJ/mole)'):
            tag = re.search(STRIP_BEFORE_PARENS, key).group(0)
            E_kj_val = state_data[key][0]
            E_kj = E_kj_val * kilojoule_per_mole
            E_kcal = E_kj.in_units_of(kilocalorie_per_mole)

            for energy in (E_kj, E_kcal):
                energies[f'{tag} ({energy.unit.get_symbol()})'] = energy._value
        
        data_dict.update(**energies)
        data_by_mol.append(data_dict)

# collate energies into DataFrame        
df = pd.DataFrame.from_records(data_by_mol)
df = df.sort_values('Molecule')

# round energy values down to desired precision
round_fn = lambda x : round(x, PRECISION)

for col_name, col in df.items():
    try:
        df[col_name] = col.apply(round_fn) # attempt to round column and replace with rounded values
    except TypeError:
        pass

# save energies to file
energy_file = omm_dir / 'energies_1_step.csv'
df.to_csv(energy_file, index=False)

## Checking for successful residue covers of newly-generated PDB Topologies

In [ ]:
chemistry = 'urethane'

mgr = PolymerManager(coll_path / chemistry)
mol_names = failed_interchanges[chemistry]['UnmatchedAtomsError']

offmols = {
    mol_name : mgr.polymers[mol_name].offmol_matched(strict=False)
        for mol_name in mol_names
}

In [ ]:
sizes = {
    mol_name : offmol.n_atoms
        for mol_name, offmol in sorted(offmols.items(), key=lambda x : x[1].n_atoms)
}

In [ ]:

pdir = mgr.polymers['urethane_6']
# pdir = mgr.polymers['NIPU_8']
# pdir.offmol_matched(strict=True)

for atom in offmol.atoms:
    if not atom.metadata['already_matched']:
        print(atom.metadata)

In [ ]:
mgr = PolymerManager(coll_path / 'NIPU')

offmols = {}
unmatched = []
for mol_name, polymer in mgr.polymers.items():
    try:
        offmols[mol_name] = polymer.offmol
    except:
        unmatched.append(mol_name)

In [ ]:
for mol_name in unmatched:
    print(mol_name)
    polymer = mgr.polymers[mol_name]
    offmol = polymer.offmol_matched(strict=False)
    
    for atom in offmol.atoms:
        if not atom.metadata['already_matched']:
            print('\t', atom.metadata)

# Experimenting with SDF files

In [ ]:
benz = Chem.MolFromSmiles('C1ccccC=1')
benz = Chem.AddHs(benz)
benz.SetDoubleProp('stuff', 3.14)
benz

In [ ]:
block2k = Chem.MolToMolFile(benz, 'test_2k.sdf')
block3k = Chem.MolToV3KMolFile(benz, 'test_3k.sdf')

In [ ]:
block2kforce = Chem.MolToMolFile(benz, 'test_2k_force.sdf', forceV3000=True)

In [ ]:
with Chem.SDWriter('test_sdw.sdf') as sdwriter:
    sdwriter.SetForceV3000(True)
    print(sdwriter.GetForceV3000())

    sdwriter.write(benz)

In [ ]:
with Chem.SDMolSupplier('sdf_testing/test_off_rd.sdf', sanitize=False) as suppl:
    mols = [mol for mol in suppl]

targ = mols[0]
targ

In [ ]:
omol = Molecule.from_rdkit(benz)
omol.generate_conformers(n_conformers=1)
omol.visualize(backend='nglview')

In [ ]:
from polysaccharide import filetree
from polysaccharide import TOOLKITS


p = Path('sdf_testing/test_off_rd.sdf')
tkwrap = TOOLKITS['The RDKit']

omol.properties['series'] = (1,2,3)
omol.to_file(
    general.asstrpath(p),
    file_format=filetree.dotless(p.suffix),
    toolkit_registry=tkwrap
)

In [ ]:
omol_load = Molecule.from_file(
    general.asstrpath(p),
    file_format=filetree.dotless(p.suffix),
    toolkit_registry=tkwrap
)